# Soft Prompt Tuning

Prompt tuning adds a small set of trainable virtual tokens (soft prompts) to the input while keeping the pre-trained model's weight frozen. 
These virtual prompts are not human-readable; they are appended to the start of a prompt to serve as a task-specific guide during LLM training or inferences.

For example, the virtual tokens are prefixed to text for sentiment classification:

```
[virtual tokens] I love Fridays!
```

where `[virtual tokens]` are the inserted embeddings. These virtual tokens can be randomly generated or initialized from a vocabulary.

During training, these token embeddings are updated while the base model remains frozen. 

In [1]:
# Import packages
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType, PeftConfig, PeftModel
from datasets import load_dataset
from torch.utils.data import DataLoader

from tqdm import tqdm

In [2]:
!pip install datasets==3.6.0 --force-reinstall

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using ca

In [3]:
model_name = "bigscience/bloomz-560m"

username = 'ought/raft'
dataset_name = "twitter_complaints"

In [4]:
# TODO: Load dataset
dataset = load_dataset(username, dataset_name)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 3399
    })
})


In [6]:
# TODO: Display dataset 
print(dataset['train'][0])
print(dataset['train'].features)
print(dataset['train'].features['Label'].names)

{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2}
{'Tweet text': Value(dtype='string', id=None), 'ID': Value(dtype='int32', id=None), 'Label': ClassLabel(names=['Unlabeled', 'complaint', 'no complaint'], id=None)}
['Unlabeled', 'complaint', 'no complaint']


In [12]:
# TODO: Add text_label from Label
# Call dataset_enh
classes = dataset['train'].features['Label'].names
print(classes[0])
dataset_enh = dataset.map(
   lambda x: { 'text_label': [ classes[label] for label in x['Label'] ]  },
   batched=True,
   num_proc=1
)

Unlabeled


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [15]:
print(dataset_enh['train'][0])

{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2, 'text_label': 'no complaint'}


In [16]:
# TODO: Load tokenizer
# call variable name tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [17]:
# Utility function to preprocess teweets

def preprocess_function(examples):
    text_column = 'Tweet text'
    label_column = 'text_label'
    max_length = 64
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
# TODO: Preprocess the Tweets
# Call preprocessed tweets dataset_processed
dataset_processed = dataset_enh.map(
   preprocess_function,
   batched=True,
   num_proc=1,
   remove_columns=dataset_enh['train'].column_names,
   load_from_cache_file=False
)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [20]:
# TODO: Explore preprocessed Tweets
print(dataset_processed['train'][0])
for k, v in dataset_processed['train'][0].items():
   print(f'k = {k}\n\tv = {v}')

{'input_ids': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 227985, 5484, 915, 2566, 169403, 15296, 36272, 525, 3928, 1119, 632, 2670, 3968, 15270, 77658, 915, 210, 1936, 106863, 3], 'attention_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 1936, 106863, 3]}
k = input_ids
	v = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

In [22]:
# TODO: Create train and evaluation dataset 
# Must call train set DataLoader loader_train
# Must call test set DataLoader loader_test

batch_size = 8

# Get the train and test set
dataset_train = dataset_processed['train']
dataset_test = dataset_processed['test']

# Create trand and test loaders
loader_train = DataLoader(dataset_train, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
loader_test = DataLoader(dataset_test, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)


## Soft Prompt 

In [23]:
# TODO: Create PEFT configuration
config = PromptTuningConfig(
   task_type=TaskType.CAUSAL_LM,
   num_virtual_tokens=8,
   tokenizer_name_or_path=model_name,
   prompt_tuning_init=PromptTuningInit.RANDOM
)

# config = PromptTuningConfig(
#    task_type=TaskType.CAUSAL_LM,
#    num_virtual_tokens=8,
#    tokenizer_name_or_path=model_name,
#    prompt_tuning_init=PromptTuningInit.TEXT,
#    prompt_tuning_init_text="big black bug bleeds black blodd"
# )

In [24]:
# TODO: Create model for training
model = AutoModelForCausalLM.from_pretrained(model_name)
# similar to LoRA
peft_model = get_peft_model(model, peft_config=config)
print(peft_model.print_trainable_parameters())

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0015
None


In [25]:
print(peft_model)

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(250880, 1024)
      (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
          )
        )
      

In [ ]:
# Create optimizer
lr = 3e-2
num_epochs = 1
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
   optimizer=optimizer,
   num_warmup_steps=0,
   num_training_steps=(len(loader_train) * num_epochs)
)

In [ ]:
# Train model
# default to CPU
device = "cuda"
device = "cpu"

model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(loader_train)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.requires_grad = True
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(loader_test)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(loader_test)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(loader_train)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

## Inferencing

In [26]:
peft_model_id = "stevhliu/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"

In [27]:
## TODO: Load trained model
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
   config.base_model_name_or_path
)
peft_model = PeftModel.from_pretrained(model, peft_model_id)


In [38]:
## TODO: Write and encode text

input_text = "Tweet text: @uni I hate the food in your canteen. Label: " 
#input_text = "Tweet text: @husband The pipe has been leaking for over one month. Can fix this NOW? Label: "
#input_text = "Tweet text: @ChatGPT You are not very good at predicting my thoughts! Label:"

input_enc = tokenizer(input_text, return_tensors='pt')

In [39]:
## TODO: Determine sentiment from model
output_enc = peft_model.generate(
   input_ids = input_enc.input_ids, 
   attention_mask=input_enc.attention_mask,
   max_new_tokens=10,
   eos_token_id=3
)

print(output_enc)

output = tokenizer.decode(output_enc[0], skip_special_tokens=True)
print(output)


tensor([[227985, 106659,   2566,   8196,    473,  74549,    368,  17304,    361,
           2632, 177137,    257,     17,  77658,     29,    210,   1936, 106863,
              3]])
Tweet text: @uni I hate the food in your canteen. Label: no complaint
